<div class="alert-success">
1) Создать скрипт для анализа тестовых файлов - сбора статистики упоминания слов. Аргументом командной строки является имя файла, который нужно проанализировать. Второй аргумент - количество слов, которые нужно выводить на экран (если аргумент не указан, то выводим статистику по 100 словам, статистика выводится в порядке убывания частоты слов, по каждому слову на экран выводится строка в формате "<слово> <частота>"). Если второй аргумент не целое число, то его рассматриваем как имя файла, в который в формате CSV с заголовком сохраняем статистику слов (Первый столбец - слово, второй столбце - частота).

In [ ]:
def script(file_name, count=100):
    # Импорт необходимых библиотек
    import csv
    import string

    # Открытие файла и чтение его содержимого
    with open(file_name, 'r', encoding='utf-8') as file:
        content = file.read()

    # Преобразование текста в нижний регистр и удаление знаков пунктуации
    content = content.lower()
    for i in string.punctuation:
        content = content.replace(i, '')
    content = content.replace('–', '')  # Замена тире на пустую строку
    words = content.split()  # Разделение содержимого файла на отдельные слова

    # Создание словаря для подсчета количества каждого слова
    word_count = {}
    for word in words:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1

    # Сортировка словаря по убыванию частоты появления слов
    sorted_word_count = dict(sorted(word_count.items(), key=lambda item: item[1], reverse=True))

    # Вывод результата на экран или запись в CSV файл в зависимости от входных параметров
    if isinstance(count, int):  # Проверка, если параметр count является целым числом
        c = 0
        for key, value in sorted_word_count.items():
            c += 1
            if c <= count:
                print(key, value)  # Вывод наиболее часто встречающихся слов в тексте
    else:
        with open(str(count) + '.csv', 'w', newline='') as csvfile:  # Создание CSV файла
            writer = csv.writer(csvfile)
            for key, value in sorted_word_count.items():
                writer.writerow([key, value])  # Запись пар слово-количество в CSV файл


if __name__ == '__main__':  # Проверка, если модуль был запущен как скрипт, а не импортирован
    import sys

    if len(sys.argv) < 2:
        print('Неверный ввод')  # Вывод сообщения об ошибке при неверном вводе
        sys.exit(1)  # Завершение программы с кодом ошибки
    filename = sys.argv[1]

    if len(sys.argv) == 3:
        try:
            num_words = int(sys.argv[2])  # Попытка привести третий аргумент к целому числу
        except ValueError:
            num_words = sys.argv[2]

        script(filename, num_words)  # Вызов функции script с указанием количества слов
    else:
        script(filename)  # Вызов функции script без указания количества слов

<div class="alert-success">
2) Создать скрипт, которому в командной строке передаются имена файлов. В файлах в формате CSV с заголовком содержится информация о частоте слов: первый столбец - слово, второй столбце - частота. Выполнить агрегацию информации о частоте слов из всех переданных файлов (т.е. рассчитать суммарную частоту для всех упомянутых слов). Результат сохранить в файл с именем, которое введет пользователь в ответ на соответствующий запрос.
2.1) Скрипт должен адекватно реагировать на ситуацию отсутствия файлов и другие проблемы с вводом/выводом.
2.2) Оформить модуль с функциями, которые используются в задании 1 и 2.

In [ ]:
import csv
import sys

# Функция для агрегирования частотности слов в нескольких файлах
def aggregate_word_frequency(input_files):
    word_freq = {}  # Инициализация словаря для хранения частотности слов
    for file_name in input_files:
        try:
            with open(file_name, 'r', newline='', encoding='utf-8') as file:
                csv_reader = csv.reader(file)
                next(csv_reader)  # Пропуск заголовка
                for row in csv_reader:
                    word = row[0]  # Слово
                    freq = int(row[1])  # Частота
                    word_freq[word] = word_freq.get(word, 0) + freq  # Обновление частотности слова
        except FileNotFoundError:
            print(f"File '{file_name}' not found.")  # Вывод сообщения об ошибке при отсутствии файла
        except Exception as e:
            print(f"Error processing file '{file_name}': {e}")  # Вывод сообщения об ошибке при обработке файла
    return word_freq  # Возвращение агрегированной частотности слов

# Функция для сохранения агрегированной частотности в файл
def save_aggregated_frequency(aggregated_freq, output_file):
    try:
        with open(output_file, 'w', newline='', encoding='utf-8') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow(['Word', 'Frequency'])  # Запись заголовка в CSV файл
            for word, freq in aggregated_freq.items():
                csv_writer.writerow([word, freq])  # Запись данных в CSV файл
            print(f"Aggregated frequency saved to '{output_file}'.")  # Вывод сообщения об успешном сохранении
    except Exception as e:
        print(f"Error saving aggregated frequency: {e}")  # Вывод сообщения об ошибке при сохранении

# Проверка, если модуль был запущен как скрипт, а не импортирован
if __name__ == "main":
    if len(sys.argv) < 2:
        print("Usage: python script.py file1.csv file2.csv ...")  # Вывод сообщения об использовании
        sys.exit(1)  # Завершение программы с кодом ошибки

    input_files = sys.argv[1:]  # Получение списка входных файлов из аргументов командной строки
    aggregated_freq = aggregate_word_frequency(input_files)  # Агрегирование частотности слов в указанных файлах

    output_file = input("Enter the name of the output file: ")  # Запрос имени выходного файла у пользователя
    save_aggregated_frequency(aggregated_freq, output_file)  # Сохранение агрегированной частотности слов в файл

<div class="alert-success">
3.1) Написать программу, которая запускается из консоли и считает сумму переданных в неё чисел
3.2) Добавить в эту программу произведение
3.3) По переданному параметру считать либо сумму, либо произведение
3.4) Оформить это дополнительный модуль.

In [ ]:
def calculate(operation, numbers):
    result = 0  # Инициализация переменной результата
    
    # Проверка выбранной операции и выполнение соответствующих вычислений
    if operation == 'сумма':
        result = sum(numbers)  # Если операция - сумма, вычисляем сумму чисел
    elif operation == 'произведение':
        result = 1  # Инициализация переменной результата для операции умножения
        for num in numbers:
            result *= num  # Если операция - произведение, вычисляем произведение чисел
    
    return result  # Возвращаем результат вычислений


if __name__ == "main":  # Проверка, если модуль был запущен как скрипт, а не импортирован
    import sys

    if len(sys.argv) < 3:  # Проверка количества переданных аргументов
        print("Неверный ввод")  # Вывод сообщения об ошибке при неверном вводе
        sys.exit(1)  # Завершение программы с кодом ошибки

    operation = sys.argv[1]  # Получение операции из аргументов командной строки
    numbers = [float(x) for x in sys.argv[2:]]  # Получение чисел из аргументов командной строки и преобразование их в тип float

    result = calculate(operation, numbers)  # Вызов функции для выполнения вычислений
    print("Результат:", result)  # Вывод результата

<div class="alert-success">
    4) Реализовать модуль calculate.py, содержащий набор функций для работы с целыми числами, записанными как на русском языке (например: тридцать восемь), так и в виде простых арифметических выражений (например: 2 + 3).
На базе этого модуля подготовить скрипт calculate.py со следующими возможностями:
1) python claculate.py to_numbers my_file.txt
Вызов приводит к преобразованию всех чисел, написанных на русском языке в числовом виде и сохранению изменений в файл с тем же именем.
python claculate.py to_numbers my_file.txt my_file_res.txt
(результат сохраняется в файл с именем my_file_res.txt)

In [19]:
import words2numsrus
import sys


def calculate(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f: # берем из файла текст
        content = f.read()
    extractor = words2numsrus.NumberExtractor()
    converted_content = extractor.replace_groups(content) # используем библиотеку преобразовываем числа
    words = converted_content.split() #преобразуем слова в цифры при этом никуда не заисываем их
    summ = int(words[0]) # первое число в примере
    for i in range(len(words) - 1): #первое число мы забрали, поэтому с 2-ого эллемента
        if words[i] == '+' or words[i].lower() == 'плюс': # проверка на плюс
            summ += int(words[i + 1])
        elif words[i] == '-' or words[i].lower() == 'минус': # проверка на минус
            summ -= int(words[i + 1])
    with open(output_file, 'w', encoding='utf-8') as f: # запись в файл
        f.write(str(summ))


def convert_to_numbers(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f: # берем из файла текст
        content = f.read()
    extractor = words2numsrus.NumberExtractor()
    converted_content = extractor.replace_groups(content) # используем библиотеку преобразовываем числа
    with open(output_file, 'w', encoding='utf-8') as f: # запись в файл
        f.write(str(converted_content))
        
        
if __name__ == "__main__":
    if len(sys.argv) < 3:
        print("Неверный ввод")
        sys.exit(1)
    input_file = sys.argv[2]
    output_file = sys.argv[3] if len(sys.argv) == 4 else sys.argv[2] # если не указан выходной файл делаем входной выходным
    if sys.argv[1] == 'to_numbers':
        convert_to_numbers(input_file, output_file)
    else:
        calculate(input_file, output_file)

TypeError: expected str, bytes or os.PathLike object, not NoneType